In [1]:
# import bus_processing
import bus_prediction
import os
import pandas as pd
from constants import DATA_DIR

/Users/legendword/python3/lib/python3.12/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [2]:
def load_bus_data():
    path = os.path.join(DATA_DIR, "dataset.csv")
    df = pd.read_csv(path)
    return df

bus_data = load_bus_data()
bus_data["time_bucket"] = pd.to_datetime(bus_data["time_bucket"], utc=True)
X_train, y_train, X_test, y_test = bus_prediction.prepare_bus_data(bus_data)

In [3]:
tft_predictor = bus_prediction.TFTPredictor()
tft_predictor.train(X_train, y_train)

/Users/legendword/python3/lib/python3.12/site-packages/pytorch_forecasting/data/encoders.py:400: UserWarning: Found 1 unknown classes which were set to NaN
  warnings.warn(
/Users/legendword/python3/lib/python3.12/site-packages/lightning/pytorch/utilities/parsing.py:209: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
/Users/legendword/python3/lib/python3.12/site-packages/lightning/pytorch/utilities/parsing.py:209: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/Users/legendword/python3/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:76: Starting 

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/Users/legendword/python3/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
/Users/legendword/python3/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


In [5]:
tft_predictor.evaluate(X_test, y_test)

Restoring states from the checkpoint path at /Users/legendword/Documents/zwh/Coding/CPSC440/cpsc440-transit-project/src/lightning_logs/version_80/checkpoints/epoch=1-step=344.ckpt
Loaded model weights from the checkpoint at /Users/legendword/Documents/zwh/Coding/CPSC440/cpsc440-transit-project/src/lightning_logs/version_80/checkpoints/epoch=1-step=344.ckpt
/Users/legendword/python3/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_MAE          │    0.15751099586486816    │
│         test_MAPE         │    0.1002342700958252     │
│         test_RMSE         │    0.15751099586486816    │
│        test_SMAPE         │    0.1055227741599083     │
│         test_loss         │    0.15751099586486816    │
└───────────────────────────┴───────────────────────────┘

In [ ]:
# import numpy as np
# def test_tft():
#     transformed_X_test = X_test.copy()
#     transformed_X_test[bus_prediction.numerical_columns] = tft_predictor.scalar.transform(transformed_X_test[bus_prediction.numerical_columns])
#     transformed_test_data = pd.concat([transformed_X_test, y_test], axis=1)
#     transformed_test_data["bus_id"] = 0
#     transformed_test_data["time_idx"] = np.arange(len(transformed_test_data))
#     transformed_test_data = transformed_test_data.drop(columns=["time_bucket"])
#     transformed_test_data[bus_prediction.categorical_columns] = transformed_test_data[bus_prediction.categorical_columns].astype(str)
#     encoder_len = 24
#     predictions = []
#     for i in range(encoder_len, len(transformed_test_data)):
#         partial_data = transformed_test_data[i - encoder_len:i]
#         output = tft_predictor.model.predict(partial_data, mode="prediction")
#         prediction = output[0][0]
#         predictions.append(prediction)
#         # print(f"Predicted: {prediction}, Actual: {transformed_test_data.iloc[i]['avg_delay']}")
        
#     predictions = np.array(predictions)
#     expected = transformed_test_data.iloc[encoder_len:]["avg_delay"].values
#     mse = np.mean((predictions - expected) ** 2)
#     print(f"Test MSE: {mse}")

# test_tft()

AttributeError: 'TFTPredictor' object has no attribute 'scalar'

In [ ]:
lstm_model = bus_prediction.LSTMModule(input_size=X_train.shape[1]-1, hidden_size=64, num_layers=5, dropout=0.1)
lstm_predictor = bus_prediction.LSTMPredictor(lstm_model, window_size=10)

In [ ]:
lstm_predictor.train(X_train, y_train, epochs=20)

/Users/legendword/Documents/zwh/Coding/CPSC440/cpsc440-transit-project/src/bus_prediction.py:55: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:281.)
  self.X = torch.tensor(X_lagged, dtype=torch.float32)


Epoch 1/20, Train Loss: 1.2768, Val Loss: 0.4806
Epoch 2/20, Train Loss: 0.9598, Val Loss: 0.4422
Epoch 3/20, Train Loss: 0.8753, Val Loss: 0.4406
Epoch 4/20, Train Loss: 0.8524, Val Loss: 0.6108
Epoch 5/20, Train Loss: 0.8444, Val Loss: 0.5659
Epoch 6/20, Train Loss: 0.8058, Val Loss: 0.5824
Epoch 7/20, Train Loss: 0.7809, Val Loss: 0.6077
Epoch 8/20, Train Loss: 0.7991, Val Loss: 0.5587
Epoch 9/20, Train Loss: 0.7481, Val Loss: 0.6745
Epoch 10/20, Train Loss: 0.7707, Val Loss: 0.5285
Epoch 11/20, Train Loss: 0.7492, Val Loss: 0.5622
Epoch 12/20, Train Loss: 0.7042, Val Loss: 0.6053
Epoch 13/20, Train Loss: 0.7321, Val Loss: 0.6767
Epoch 14/20, Train Loss: 0.7093, Val Loss: 0.5340
Epoch 15/20, Train Loss: 0.7308, Val Loss: 0.7020
Epoch 16/20, Train Loss: 0.6774, Val Loss: 0.6309
Epoch 17/20, Train Loss: 0.6512, Val Loss: 0.7870
Epoch 18/20, Train Loss: 0.6832, Val Loss: 0.8902
Epoch 19/20, Train Loss: 0.6252, Val Loss: 0.4964
Epoch 20/20, Train Loss: 0.6214, Val Loss: 0.4286


In [ ]:
mse = lstm_predictor.evaluate(X_test, y_test)
print(f"Test MSE: {mse:.4f}")

Shape of y_pred: (2192,)
Shape of test_y: (2192,)
Test MSE: 1.0221


In [ ]:
xgboost_predictor = bus_prediction.XGBoostPredictor()
xgboost_predictor.train(X_train, y_train)
xgboost_mse = xgboost_predictor.evaluate(X_test, y_test)
print(f"XGBoost Test MSE: {xgboost_mse:.4f}")

Validation MSE: 0.2797
Shape of y_pred: (2202,)
Shape of test_y: (2202,)
XGBoost Test MSE: 0.2485
